<a href="https://colab.research.google.com/github/akhilsaboo05/public/blob/main/Phase_1_%26_2_SAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import logging

In [2]:
class Phase1:

    def __init__(self):

        self.Events = pd.read_excel('Events.xlsx')

        self.dist = pd.read_excel('distance_mat_temp.xlsx',index_col=0)

    def has_primary_interests(self, users_interest, events_interest):

        count = 0

        for tag in users_interest:

            if tag in events_interest:

                count += 2

        return count

    def has_secondary_interests(self, users_interest, events_interest):

        count = 0

        for tag in users_interest:

            if tag in events_interest:

                count += 1

        return count


    def get_count(self, users_interest):
        self.Events['Primary Count'] = self.Events['Primary Category'].apply(lambda x: self.has_primary_interests(users_interest, x))
        #primary_count = (self.Events['Primary Count']==1).sum()
        clean_secondary_category = self.Events['Secondary Category'].fillna("None")
        #print(clean_secondary_category)
        self.Events['Secondary Count'] = clean_secondary_category.apply(lambda x: self.has_secondary_interests(users_interest, x))
        #secondary_count = (self.Events['Secondary Count']==1).sum()
        self.Events['Count'] = self.Events['Primary Count'] + self.Events['Secondary Count']

        '''num_interests = len(users_interest)

        if num_interests > 10:

            num_events_per_interest = 1

        else:

            num_events_per_interest = 10 // num_interests

        events_per_category = {}

        for interest in users_interest:

            events_per_category[interest] = num_events_per_interest

        remaining_events  = 10-(num_interests*num_events_per_interest)

        if remaining_events > 0:

            for i in range(remaining_events):

                idx = i % num_interests

                interest = users_interest[idx]

                events_per_category[interest] += 1'''

        selected_events = []

        selected_event_ids = set()

        for interest in users_interest:

            events_sorted = self.Events[self.Events['Primary Category'].apply(lambda x: interest in x.split(','))].sort_values('Count', ascending=False)

            count = 0

            for index,event in events_sorted.iterrows():

                if event['Event ID'] not in selected_event_ids:

                    selected_events.append(event)

                    selected_event_ids.add(event['Event ID'])

                    count += 1

                    '''if count == num_events:

                        break'''

        selected_events_df = pd.DataFrame(selected_events)

        self.Events = pd.concat([selected_events_df])


    def get_distance(self, city):

        def get_distance_or_default(location):
            try:
                return self.dist.loc[city][location]
            except KeyError:
                return 10.0  # default distance of 10 km

        self.Events['Distance'] = self.Events['Location'].apply(get_distance_or_default)

    def predict(self, users_interest, city):

        self.get_count(users_interest)

        self.get_distance(city)

        self.Events = self.Events.sort_values(by=['Count','Distance'],ascending = [False,True])

        self.Events = self.Events[['Event ID','Event Names','Primary Category','Location','Prices','Start Date', 'End Date', 'Start Time', 'End Time','Count','Distance']]

        return self.Events

In [3]:
#!pip install recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.0/356.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.6/749.6 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 18.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.9/251.9 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.8/919.8 kB 49.4 MB/s eta 0:00:00
     ━━━

In [87]:
events = pd.read_excel('Events.xlsx')

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.models.sar import SAR
from recommenders.utils.python_utils import binarize
from sklearn.preprocessing import minmax_scale
from recommenders.evaluation.python_evaluation import (
    map,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    rmse,
    mae,
    logloss,
    rsquared,
    exp_var
)

In [5]:
TOP_K = 50

In [16]:
combined_interactions = pd.read_excel("Combined_Interactions.xlsx")
combined_interactions["Interaction Score"] = combined_interactions["Interaction Score"].astype(np.float32)

# model can't interpret negative numbers, so shift all scores up by 1
min_score = combined_interactions["Interaction Score"].min()
combined_interactions["Interaction Score"] += abs(min_score)

combined_interactions

,Name,Event ID,Interaction,Interaction Score,Primary Category,Secondary Category,Sub Category
0,Aakash,EVT0004,Disliked,0.0,Travel,NaN,One Day Outing
1,Aakash,EVT0007,Booked,6.0,Adventure,NaN,Adventure Sports
2,Aakash,EVT0014,Create Circle,8.0,Travel,Adventure,Camping
3,Aakash,EVT0015,Join Circle,7.0,Adventure,Travel,Hiking and Trekking
4,Aakash,EVT0018,Booked,6.0,Tech and Innovation,NaN,Exhibition
...,...,...,...,...,...,...,...
335,Shreyas2,EVT0158,Skipped,1.0,Music and Entertainment,NaN,Comedy
336,Shreyas2,EVT0005,Skipped,1.0,Music and Entertainment,NaN,"Poetry,Comedy"
337,Shreyas2,EVT0197,Skipped,1.0,Music and Entertainment,NaN,Concert
338,Shreyas2,EVT0084,Skipped,1.0,Music and Entertainment,NaN,Instrument


In [17]:
train, test = python_stratified_split(combined_interactions, ratio=0.75, col_user="Name", col_item="Event ID", seed = 153)

In [18]:
print("""
Train:
Total Interactions: {train_total}
Unique Users: {train_users}
Unique Events: {train_events}

Test:
Total Interactions: {test_total}
Unique Users: {test_users}
Unique Events: {test_events}
""".format(
    train_total=len(train),
    train_users=len(train['Name'].unique()),
    train_events=len(train['Event ID'].unique()),
    test_total=len(test),
    test_users=len(test['Name'].unique()),
    test_events=len(test['Event ID'].unique()),
))


Train:
Total Interactions: 258
Unique Users: 20
Unique Events: 74

Test:
Total Interactions: 82
Unique Users: 20
Unique Events: 42



In [19]:
model = SAR(
    col_user="Name",
    col_item="Event ID",
    col_rating="Interaction Score",
    similarity_type="jaccard",
    time_decay_coefficient=30,
    timedecay_formula=False,
    normalize=True
)

In [20]:
model.fit(train)

In [21]:
top_k = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_data.py:139: RuntimeWarning: invalid value encountered in multiply
  return self._with_data(self.data * other)


In [22]:
top_k

,Name,Event ID,prediction
0,Aakash,EVT0025,3.355277
1,Aakash,EVT0085,3.310801
2,Aakash,EVT0010,3.093272
3,Aakash,EVT0023,2.911114
4,Aakash,EVT0147,2.714777
...,...,...,...
992,Vrushali,EVT0148,0.686503
993,Vrushali,EVT0028,0.674382
994,Vrushali,EVT0017,0.656725
995,Vrushali,EVT0129,0.532042


In [ ]:
sujan_interests = 'Sports, Tech and Innovation, Music and Entertainment, Books and Education, Adventure, Fitness, Travel, Virtual Events, Photography, Food and Drinks, Fashion'
split_sujan_interests = sujan_interests.split(', ')
split_sujan_interests

['Sports',
 'Tech and Innovation',
 'Music and Entertainment',
 'Books and Education',
 'Adventure',
 'Fitness',
 'Travel',
 'Virtual Events',
 'Photography',
 'Food and Drinks',
 'Fashion']

In [26]:
responses = pd.read_excel('Responses.xlsx')
responses

,Timestamp,Name,Unnamed: 2,Unnamed: 3,Gender,Age,Interests (Multi Select),Residential Location (Area)
0,2024-03-27 18:52:41.085,Shreyas,Saboo,NaN,Male,24,"Sports, Tech and Innovation, Adventure, Travel...",Koregaon Park
1,2024-03-27 18:49:31.567,Aakash,Soni,NaN,Male,26,"Sports, Tech and Innovation, Business and Netw...",Hadapsar
2,2024-03-27 18:49:51.970,Ruma,NaN,NaN,Female,24,"Sports, Adventure, Travel, Food and Drinks",Hinjawadi
3,2024-03-27 18:49:52.137,ankit,parsana,NaN,Male,33,"Sports, Tech and Innovation, Adventure, Fitnes...",Shivajinagar
4,2024-03-27 18:50:15.941,Chinmay,Joshi,NaN,Male,27,"Tech and Innovation, Music and Entertainment, ...",Sinhagad Road
5,2024-03-27 18:50:28.438,Rupesh,NaN,NaN,Male,30,"Sports, Tech and Innovation, Business and Netw...",Dighi
6,2024-03-27 18:50:37.962,Vrushali,NaN,NaN,Female,27,"Sports, Music and Entertainment, Dance, Art an...",Bhugaon
7,2024-03-27 18:51:32.396,Priyanka,Bangarwa,NaN,Female,21,"Sports, Music and Entertainment, Dance, Advent...",Koregaon Park
8,2024-03-27 18:52:55.868,Avinash,Jagtap,NaN,Male,27,"Adventure, Fitness",Bhosari
9,2024-03-27 18:53:46.057,Dinesh,Kumar,NaN,Male,22,"Sports, Tech and Innovation, Music and Enterta...",Swargate


In [27]:
user = Phase1()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
user = Phase1()
user_responses = responses[responses['Name']=='Sujan']
user_interests = user_responses['Interests (Multi Select)']
user_interests = user_interests.values[0].split(', ')
user_interests
user_location = user_responses['Residential Location (Area)']
user_location = user_location.values[0]
user_location
user.predict(user_interests, user_location)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Event ID,Event Names,Primary Category,Location,Prices,Start Date,End Date,Start Time,End Time,Count,Distance
144,EVT0147,Flea Market - local shopping,Music and Entertainment,Karve Nagar,50,2024-04-13,2024-04-13,17:00:00,19:10:00,3,2.300456
21,EVT0022,Fun and Fair,Music and Entertainment,Erandwane,0,2024-03-23,2024-03-31,15:00:00,15:30:00,3,2.512520
78,EVT0081,Kolad River Side Camping and Water Sports,Adventure,Deccan,1500,2024-03-31,2024-03-31,15:00:00,17:00:00,3,3.221632
117,EVT0120,Saturday Mix Night,Music and Entertainment,Shivajinagar,1300,2024-03-31,2024-03-31,20:00:00,22:30:00,3,4.941765
175,EVT0178,Obstacle Course - Pune,Sports,Pune,0,2024-06-02,2024-06-02,07:00:00,07:40:00,3,5.235846
...,...,...,...,...,...,...,...,...,...,...,...
32,EVT0033,Panshet Water Sports,Adventure,Kharadi,599,2024-10-04,2024-10-04,07:00:00,08:30:00,2,15.652623
49,EVT0051,Nature Explorer’s Day,Travel,Kharadi,2500,2024-07-10,2024-07-10,20:00:00,21:00:00,2,15.652623
17,EVT0018,Global Tech Engineering Expo,Tech and Innovation,Ravet,0,2024-03-30,2024-03-30,16:00:00,18:10:00,2,16.186049
40,EVT0041,Cricket Tournament,Sports,Wagholi,600,2024-11-12,2024-11-12,10:00:00,11:00:00,2,19.803190


In [28]:
def getUserEvents(name):
    user = Phase1()
    user_responses = responses[responses['Name']== name]
    user_interests = user_responses['Interests (Multi Select)']
    user_interests = user_interests.values[0].split(', ')
    user_interests
    user_location = user_responses['Residential Location (Area)']
    user_location = user_location.values[0]
    return user.predict(user_interests, user_location)

In [32]:
sujanEvents = getUserEvents('Sujan')

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
top_k[top_k['Name'] == 'Sujan']

,Name,Event ID,prediction
750,Sujan,EVT0085,3.075043
751,Sujan,EVT0018,2.950207
752,Sujan,EVT0007,2.737735
753,Sujan,EVT0147,2.654634
754,Sujan,EVT0030,2.540469
755,Sujan,EVT0029,2.419412
756,Sujan,EVT0004,2.190980
757,Sujan,EVT0040,2.020816
758,Sujan,EVT0064,1.822718
759,Sujan,EVT0100,1.768302


In [ ]:
threshold = 2
phase2_filtered = top_k[top_k['prediction'] >= threshold]
phase2_filtered = phase2_filtered.drop(columns=['prediction'])
sujan_threshold = phase2_filtered[phase2_filtered['Name'] == 'Sujan'].drop(columns=['Name'])
sujan_threshold = sujan_threshold.reset_index(drop=True)
sujan_threshold

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
5,EVT0029
6,EVT0004
7,EVT0040


In [34]:
sujan_phase1_filtered = sujanEvents.iloc[:, :1]
sujan_phase1_filtered = sujan_phase1_filtered.reset_index(drop=True)
sujan_phase1_filtered

,Event ID
0,EVT0147
1,EVT0022
2,EVT0081
3,EVT0120
4,EVT0014
...,...
135,EVT0017
136,EVT0033
137,EVT0051
138,EVT0018


In [ ]:
# recommend first the phase 2 results over the threshold, then the rest of the unique phase 1 results
pd.concat([sujan_threshold,sujan_phase1_filtered],axis=0).drop_duplicates(subset='Event ID')

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
...,...
148,EVT0193
149,EVT0017
151,EVT0051
153,EVT0041


In [ ]:
threshold = 1.5
phase2_filtered = top_k[top_k['prediction'] >= threshold]
phase2_filtered = phase2_filtered.drop(columns=['prediction'])
sujan_threshold = phase2_filtered[phase2_filtered['Name'] == 'Sujan'].drop(columns=['Name'])
sujan_threshold = sujan_threshold.reset_index(drop=True)
sujan_threshold

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
5,EVT0029
6,EVT0004
7,EVT0040
8,EVT0064
9,EVT0100


In [ ]:
pd.concat([sujan_threshold,sujan_phase1_filtered],axis=0).drop_duplicates(subset='Event ID')

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
...,...
148,EVT0193
149,EVT0017
151,EVT0051
153,EVT0041


In [ ]:
threshold = 1
phase2_filtered = top_k[top_k['prediction'] >= threshold]
phase2_filtered = phase2_filtered.drop(columns=['prediction'])
sujan_threshold = phase2_filtered[phase2_filtered['Name'] == 'Sujan'].drop(columns=['Name'])
sujan_threshold = sujan_threshold.reset_index(drop=True)
sujan_threshold

NameError: name 'top_k' is not defined

In [ ]:
pd.concat([sujan_threshold,sujan_phase1_filtered],axis=0).drop_duplicates(subset='Event ID')

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
...,...
147,EVT0031
148,EVT0193
149,EVT0017
151,EVT0051


In [ ]:
threshold = 0.5
phase2_filtered = top_k[top_k['prediction'] >= threshold]
phase2_filtered = phase2_filtered.drop(columns=['prediction'])
sujan_threshold = phase2_filtered[phase2_filtered['Name'] == 'Sujan'].drop(columns=['Name'])
sujan_threshold = sujan_threshold.reset_index(drop=True)
sujan_threshold

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
5,EVT0029
6,EVT0004
7,EVT0040
8,EVT0064
9,EVT0100


In [ ]:
pd.concat([sujan_threshold,sujan_phase1_filtered],axis=0).drop_duplicates(subset='Event ID')

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
...,...
146,EVT0154
147,EVT0031
148,EVT0193
151,EVT0051


In [ ]:
shreyas_events = 'Sports, Tech and Innovation, Music and Entertainment, Business and Networking, Adventure, Travel, Photography, Food and Drinks'
shreyas_events.split(', ')

['Sports',
 'Tech and Innovation',
 'Music and Entertainment',
 'Business and Networking',
 'Adventure',
 'Travel',
 'Photography',
 'Food and Drinks']

In [ ]:
shreyas = Phase1()
shreyas_phase1 = shreyas.predict(shreyas_events.split(', '), 'Koregaon Park')
file_path = 'shreyas_phase1.xlsx'
shreyas_phase1.to_excel(file_path, index=False)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [54]:
shreyasEvents = getUserEvents('Shreyas2')

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Event ID,Event Names,Primary Category,Location,Prices,Start Date,End Date,Start Time,End Time,Count,Distance
82,EVT0085,Dandeli Jungle Tour,Adventure,Sangamvadi,10000,2024-03-29,2024-03-29,00:00:00,01:50:00,3,1.397922
74,EVT0077,Stargazing Party,Music and Entertainment,Yerawada,2300,2024-03-30,2024-03-30,17:00:00,17:50:00,3,2.385454
63,EVT0065,Poona Bazar Summer Edition,Music and Entertainment,Viman Nagar,2000,2024-11-29,2024-11-29,20:00:00,22:30:00,3,4.115834
24,EVT0025,Shopping Festival,Music and Entertainment,Magarpatta,0,2024-03-29,2024-04-04,11:00:00,12:30:00,3,4.169002
117,EVT0120,Saturday Mix Night,Music and Entertainment,Shivajinagar,1300,2024-03-31,2024-03-31,20:00:00,22:30:00,3,4.946537
...,...,...,...,...,...,...,...,...,...,...,...
89,EVT0092,Calling all Punekars (Join Top Unicorns),Tech and Innovation,Pimpri Chinchwad,0,2024-07-15,2024-07-15,12:00:00,14:00:00,2,14.121343
120,EVT0123,Blog while Travelling,Travel,Pimpri Chinchwad,200,2024-04-13,2024-04-13,19:00:00,20:50:00,2,14.121343
17,EVT0018,Global Tech Engineering Expo,Tech and Innovation,Ravet,0,2024-03-30,2024-03-30,16:00:00,18:10:00,2,19.199912
25,EVT0026,Musical Band and Orchestra,Music and Entertainment,Paud Road,2000,2024-03-08,2024-03-08,19:00:00,21:10:00,2,20.134948


In [68]:
shreyasEvents

,Event ID,Event Names,Primary Category,Location,Prices,Start Date,End Date,Start Time,End Time,Count,Distance
82,EVT0085,Dandeli Jungle Tour,Adventure,Sangamvadi,10000,2024-03-29,2024-03-29,00:00:00,01:50:00,3,1.397922
74,EVT0077,Stargazing Party,Music and Entertainment,Yerawada,2300,2024-03-30,2024-03-30,17:00:00,17:50:00,3,2.385454
63,EVT0065,Poona Bazar Summer Edition,Music and Entertainment,Viman Nagar,2000,2024-11-29,2024-11-29,20:00:00,22:30:00,3,4.115834
24,EVT0025,Shopping Festival,Music and Entertainment,Magarpatta,0,2024-03-29,2024-04-04,11:00:00,12:30:00,3,4.169002
117,EVT0120,Saturday Mix Night,Music and Entertainment,Shivajinagar,1300,2024-03-31,2024-03-31,20:00:00,22:30:00,3,4.946537
...,...,...,...,...,...,...,...,...,...,...,...
89,EVT0092,Calling all Punekars (Join Top Unicorns),Tech and Innovation,Pimpri Chinchwad,0,2024-07-15,2024-07-15,12:00:00,14:00:00,2,14.121343
120,EVT0123,Blog while Travelling,Travel,Pimpri Chinchwad,200,2024-04-13,2024-04-13,19:00:00,20:50:00,2,14.121343
17,EVT0018,Global Tech Engineering Expo,Tech and Innovation,Ravet,0,2024-03-30,2024-03-30,16:00:00,18:10:00,2,19.199912
25,EVT0026,Musical Band and Orchestra,Music and Entertainment,Paud Road,2000,2024-03-08,2024-03-08,19:00:00,21:10:00,2,20.134948


In [69]:
shreyas_interactions = combined_interactions[combined_interactions['Name'] == 'Shreyas2']

,Name,Event ID,Interaction,Interaction Score,Primary Category,Secondary Category,Sub Category,random
309,Shreyas2,EVT0085,Liked,4.0,Adventure,Travel,Hiking and Trekking,0.691510
310,Shreyas2,EVT0077,Booked,6.0,Music and Entertainment,Travel,Tour,0.368833
311,Shreyas2,EVT0065,Booked,6.0,Music and Entertainment,Food and Drinks,Exhibition,0.462629
312,Shreyas2,EVT0025,Skipped,1.0,Music and Entertainment,Food and Drinks,Exhibition,0.604014
313,Shreyas2,EVT0120,Skipped,1.0,Music and Entertainment,Food and Drinks,Party,0.853522
314,Shreyas2,EVT0081,Opened,2.0,Adventure,Travel,Adventure Sports,0.883845
315,Shreyas2,EVT0022,Skipped,1.0,Music and Entertainment,Food and Drinks,Exhibition,0.280197
316,Shreyas2,EVT0147,Opened,2.0,Music and Entertainment,Food and Drinks,Shopping,0.813331
317,Shreyas2,EVT0015,Opened,2.0,Adventure,Travel,Hiking and Trekking,0.087461
318,Shreyas2,EVT0014,Opened,2.0,Travel,Adventure,Camping,0.323699


In [31]:
top_k[top_k['Name'] == 'Shreyas2']

,Name,Event ID,prediction
750,Shreyas2,EVT0007,0.561945
751,Shreyas2,EVT0023,0.460144
752,Shreyas2,EVT0004,0.456542
753,Shreyas2,EVT0016,0.414572
754,Shreyas2,EVT0040,0.401311
755,Shreyas2,EVT0003,0.396183
756,Shreyas2,EVT0081,0.378603
757,Shreyas2,EVT0142,0.378559
758,Shreyas2,EVT0061,0.378454
759,Shreyas2,EVT0036,0.367308


In [72]:
#shreyas_phase1_filtered = shreyasEvents.iloc[:, :1].reset_index(drop=True)
# Merge dataframes to identify matching rows
merged_df = pd.merge(shreyasEvents, shreyas_interactions, on='Event ID', how='inner')

event_ids_to_remove = shreyas_interactions['Event ID'].unique()

# Filter out rows in shreyasEvents that have these Event IDs
shreyas_events_filtered = shreyasEvents[~shreyasEvents['Event ID'].isin(event_ids_to_remove)]

shreyas_events_filtered

,Event ID,Event Names,Primary Category,Location,Prices,Start Date,End Date,Start Time,End Time,Count,Distance
180,EVT0183,Jubin Nautiyal Live in Pune,Music and Entertainment,Pune,999,2024-05-18,2024-05-18,17:00:00,18:40:00,2,4.529025
181,EVT0184,Late Night Comedy Show,Music and Entertainment,Pune,149,2024-06-13,2024-06-20,22:00:00,23:40:00,2,4.529025
15,EVT0016,Theatre Shows,Music and Entertainment,Pune,0,2024-09-11,2024-09-12,15:30:00,17:10:00,2,4.529025
79,EVT0082,Ravi Gupta Live,Music and Entertainment,Pune,500,2024-04-06,2024-04-06,16:00:00,18:20:00,2,4.529025
80,EVT0083,Join Harsh Gujral Live,Music and Entertainment,Pune,700,2024-04-07,2024-04-07,17:00:00,19:20:00,2,4.529025
...,...,...,...,...,...,...,...,...,...,...,...
89,EVT0092,Calling all Punekars (Join Top Unicorns),Tech and Innovation,Pimpri Chinchwad,0,2024-07-15,2024-07-15,12:00:00,14:00:00,2,14.121343
120,EVT0123,Blog while Travelling,Travel,Pimpri Chinchwad,200,2024-04-13,2024-04-13,19:00:00,20:50:00,2,14.121343
17,EVT0018,Global Tech Engineering Expo,Tech and Innovation,Ravet,0,2024-03-30,2024-03-30,16:00:00,18:10:00,2,19.199912
25,EVT0026,Musical Band and Orchestra,Music and Entertainment,Paud Road,2000,2024-03-08,2024-03-08,19:00:00,21:10:00,2,20.134948


In [96]:
#thresholds = [2, 1.5, 1, 0.5, 0.25]

#threshold as 2
threshold = 2

phase2_filtered = top_k[top_k['prediction'] >= threshold].drop(columns=['prediction'])

shreyas_threshold = phase2_filtered[phase2_filtered['Name'] == 'Shreyas2'].drop(columns=['Name']).reset_index(drop=True)

final_df = pd.concat([shreyas_threshold,shreyas_events_filtered],axis=0).drop_duplicates(subset='Event ID').iloc[:, :1]

# add all the other columns to the dataframe by merging on event id (matching up the event ids)
final_df = pd.merge(final_df, events, on='Event ID', how='left')

final_df.to_excel('shreyas_final1.xlsx', index=False)

In [97]:
#threshold as 1.5
threshold = 1.5

phase2_filtered = top_k[top_k['prediction'] >= threshold].drop(columns=['prediction'])

shreyas_threshold = phase2_filtered[phase2_filtered['Name'] == 'Shreyas2'].drop(columns=['Name']).reset_index(drop=True)

final_df = pd.concat([shreyas_threshold,shreyas_events_filtered],axis=0).drop_duplicates(subset='Event ID').iloc[:, :1]

# add all the other columns to the dataframe by merging on event id (matching up the event ids)
final_df = pd.merge(final_df, events, on='Event ID', how='left')

final_df.to_excel('shreyas_final2.xlsx', index=False)

In [98]:
#threshold as 1
threshold = 1

phase2_filtered = top_k[top_k['prediction'] >= threshold].drop(columns=['prediction'])

shreyas_threshold = phase2_filtered[phase2_filtered['Name'] == 'Shreyas2'].drop(columns=['Name']).reset_index(drop=True)

final_df = pd.concat([shreyas_threshold,shreyas_events_filtered],axis=0).drop_duplicates(subset='Event ID').iloc[:, :1]

# add all the other columns to the dataframe by merging on event id (matching up the event ids)
final_df = pd.merge(final_df, events, on='Event ID', how='left')

final_df.to_excel('shreyas_final3.xlsx', index=False)

In [99]:
#threshold as 0.5
threshold = 0.5

phase2_filtered = top_k[top_k['prediction'] >= threshold].drop(columns=['prediction'])

shreyas_threshold = phase2_filtered[phase2_filtered['Name'] == 'Shreyas2'].drop(columns=['Name']).reset_index(drop=True)

final_df = pd.concat([shreyas_threshold,shreyas_events_filtered],axis=0).drop_duplicates(subset='Event ID').iloc[:, :1]

# add all the other columns to the dataframe by merging on event id (matching up the event ids)
final_df = pd.merge(final_df, events, on='Event ID', how='left')

final_df.to_excel('shreyas_final4.xlsx', index=False)

In [100]:
#threshold as 0.25
threshold = 0.25

phase2_filtered = top_k[top_k['prediction'] >= threshold].drop(columns=['prediction'])

shreyas_threshold = phase2_filtered[phase2_filtered['Name'] == 'Shreyas2'].drop(columns=['Name']).reset_index(drop=True)

final_df = pd.concat([shreyas_threshold,shreyas_events_filtered],axis=0).drop_duplicates(subset='Event ID').iloc[:, :1]

# add all the other columns to the dataframe by merging on event id (matching up the event ids)
final_df = pd.merge(final_df, events, on='Event ID', how='left')

final_df.to_excel('shreyas_final5.xlsx', index=False)